# Recurso para graficar areas de clasificación

Es necesario importar este recurso para poder visualizar. En caso de error entrar al link del repo y seguir los pasos para la instalación

In [ ]:
import requests
from pathlib import Path 
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)
from helper_functions import plot_predictions, plot_decision_boundary

# Perceptrón multicapa

**Autor:** _Benjamin Torres_

Marzo 2019

En esta práctica implementarás distintos modelos de perceptrón multicapa para clasificar datos haciendo uso de la biblioteca Pytorch.

Puedes encontrar la documentación de esta biblioteca en :
[pytorch.org](https://pytorch.org/docs/stable/index.html)

Escencialmente, PyTorch se encarga de realizar operaciones sobre **tensores**, los cuales pueden crearse haciendo uso de torch.Tensor y pasando como argumento una lista de Python, por ejemplo:

```
import torch
a = torch.Tensor([[1,0,0],[0,1,0],[0,0,1]])
```
Crea una matriz identidad de 3x3.

Podemos ejecutar operaciones con estos tensores de manera similar a como trabajamos con arreglos de Numpy, por ejemplo: 

*   Suma, con el método add() o haciendo uso del operador +
*   Producto cruz, con el método dot() o con el operador @ 
*   Producto entrada por entrada con el operador *
*   Valor máximo, con argmax()
*   Cambio de forma con view( (new shape) )

Entre las ventajas más importantes de hacer uso de esta biblioteca está la capacidad que tiene para realizar **diferenciación automática**, que permite obtener gradientes de expresiones fácilmente, lo cual es especialmente importante en redes neuronales.

In [ ]:
import math
import random
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
#!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
from tqdm import tnrange

Para la implementación de redes neuronales podemos hacer uso de la clase nn.Module.  Ésta se debe extender y se implementa el constructor, así como la manera de hacer *feed forward* con valores de entrada.  Las capas de la red se definen utilizando objetos nn.Linear.

## Ejemplo: Perceptrón simple

En la siguiente celda puedes encontrar un ejemplo de la implementación de un perceptrón que simula la compuerta AND.

In [ ]:
class Perceptron(nn.Module):
    def __init__(self, input_size, output_size, bias=True):
      ''' Puedes definir tantos parámetros de entrada como necesites, en este caso únicamente
      el número de elementos de entrada y de salida, observa que debes agregar a self los 
      objetos nn.Linear que desees usar, en este caso sólo es uno.
      '''
      super(Perceptron, self).__init__()
      self.fc1 = nn.Linear(in_features = input_size, out_features = output_size, bias=bias)

    def forward(self, inputX):
      '''Debes sobreescibir el método forward, el cual recibe únicamente una entrada:
      los valores sobre los que se evaluará la red.
      Para hacer pasar la entrada a través de una capa, sólo debes llamarla con la entrada
      como parámetro.
      Finalmente puedes notar que se puede aplicar una función de activación a
      todos los valores resultantes.
      Este método debe resultar en, al menos una salida.
      '''
      out = torch.sigmoid(self.fc1(inputX))
      return out

Con la clase implementada ahora sólo es necesario definir la manera en la que se realiza el entrenamiento.

Puedes implementar un método train que reciba los parámetros necesarios, entre los cuales se encuentra:

*   El modelo que se desea entrenar (debe heredar de nn.Module).
*   El número de epocas de entrenamiento.
*   Los datos de entrenamiento (entradas) y sus etiquetas.
*   El criterio de oprimización, es decir, la función de costo, las cuales puedes consultar en [Pytorch loss functions.](https://pytorch.org/docs/stable/nn.html#loss-functions)
*   El método de optimizacion, por ejemplo SGD, distintos algoritmos se encuentran implementados en el modulo [torch.optim](https://pytorch.org/docs/stable/optim.html)

In [ ]:
def train(net, epochs, data, labels, criterion, optimizer, cuda=False):
    '''Entrena la red net, por un numero de epocas "epochs",
    usando como función de pérdida la definida en "criterion" y el
    optimizador pasado como parámetro.'''
    avg_loss = torch.Tensor()
    tqdm_epochs = tnrange(epochs)
    for epoch in tqdm_epochs:
        for d,label in zip(data,labels):
            if(cuda and torch.cuda.is_available()): #Si nuestra PC cuenta con GPU,realizamos cálculos en ella
                d = d.cuda()
                label = label.cuda()
    
            optimizer.zero_grad() #limpiamos los gradientes actuales
            output = net(d)       #llamar a nuestro objeto red con parámetros es introducirlos a la red
            #print("o", output)
            #print("l", label)
            loss = criterion(output, label) #calculamos el error de nuestro modelo   
            loss.backward()       #calculamos el gradiente del error y se almacena dentro del modelo
            optimizer.step()      #usando los datos almacenados y el método seleccionado actualizamos los parámetros de la red

            avg_loss = torch.cat([avg_loss, torch.Tensor([loss])],0)

    tqdm_epochs.set_description("Loss %.6f"%(avg_loss.sum()/avg_loss.numel()).item())  

Para nuestro ejemplo podemos tratar el problema de aprender la compuerta AND como un problema de regresión, para lo cual puedes usar la función de error min squared error (MSE), que se encuentra implementada como MSELoss().

Además podemos utilizar descenso por el gradiente, que puedes encontrar en torch.optim.SGD. A este objeto necesitas
pasarle como argumento los parámetros (pesos) del modelo a optimizar, lo cual puedes lograr con el método parameters() de los objetos que heredan de nn.Module; en el caso del descenso por el gradiente tambien debes especificar la taza de aprendizaje.

Puedes ejecutar el entrenamiento sobre el modelo de 2 entradas y una única salida pasándolo como argumento al método train.

In [ ]:
# Entradas
X_AND = torch.Tensor([[0,0],[0,1],[1,0],[1,1]])
# Etiquetas
Y_AND = torch.Tensor([0,0,0,1])
Perceptron_AND = Perceptron(2,1)
criterio  = nn.MSELoss()
optimizer = torch.optim.SGD(Perceptron_AND.parameters(), lr=0.1)
train(Perceptron_AND, 10000, X_AND, Y_AND, criterio, optimizer)
#train(Perceptron_AND, 2, X_AND, Y_AND, criterio, optimizer)

Ahora para realizar predicciones con el modelo obtenido unicamente debes pasar los datos como parámetro a la red, la cual ejecutará de manera automática el método forward.
Dado que el problema fue modelado como regresión observarás que los primeros 3 valores son muy cercanos a 0 y el último tiene un valor muy cercano a 1.

In [ ]:
resultados = Perceptron_AND(X_AND)
print(resultados)

## Dos perceptrones en una capa

Si quisiéramos resolverlo como un problema de clasificación debemos realizar dos cambios:

*   Cambiar el formato de las etiquetas para que se encuentren en [one-hot](https://en.wikipedia.org/wiki/One-hot).  El primer perceptrón representará al 0 y el segundo, al 1.
*   Usar una función de costo que evalúe el error de clasificación, como la entropía cruzada binaria, implementada en PyTorch como BCELoss().

In [ ]:
Y_one_hot = torch.Tensor([[1,0],[1,0],[1,0],[0,1]])
Perceptron_AND_C = Perceptron(2,2)
criterio_clasificacion =  nn.BCELoss()
optimizer = torch.optim.SGD(Perceptron_AND_C.parameters(), lr=0.1)
train(Perceptron_AND_C, 50000, X_AND, Y_one_hot, criterio_clasificacion, optimizer)

Nota que el resultado será alimentado a la funcion Softmax para que puedas interpretar los resultados como probabilidades.  La salida con el valor más alto es la clase ganadora.

In [ ]:
resultados_clasificacion = Perceptron_AND_C(X_AND)
print(nn.functional.softmax(resultados_clasificacion,dim=0))

## Perceptrón multicapa

### Ejercicio 1: XOR

Haciendo uso de un perceptrón multicapa (MLP, por sus siglas en inglés) crea un modelo capaz de simular la compuerta logica XOR considerandolo un problema de regresión.

Para esta tarea debes usar tres capas:
* La capa de entrada (los valores de entrada a la compuerta)
* Una capa oculta con 3 unidades
* Una capa de salida con una unica neurona 

Deberas especificar una **taza de aprendizaje** adecuada así como el número de iteraciones necesario para lograr el aprendizaje.

In [ ]:
X_XOR = torch.Tensor([[0,0],
                      [0,1],
                      [1,0],
                      [1,1]])
Y_XOR = torch.Tensor([0,1,1,0])

In [ ]:
class XOR(nn.Module):
    def __init__(self, input_layer_size, hidden_layer_size, output_layer_size, bias=True):
        super(XOR,self).__init__()
        self.hidden = nn.Linear(input_layer_size, hidden_layer_size, bias = bias)
        self.out = nn.Linear(hidden_layer_size, output_layer_size, bias = bias)
        self.act_func = nn.Tanh()

    def forward(self, x):
      x = self.hidden(x)
      x = self.act_func(x)
      x = self.out(x)
      x = self.act_func(x)
      return x

In [ ]:
XORNet = XOR(2,3,1)
optimizer  = torch.optim.Adam(XORNet.parameters(), lr=0.1)
criterio = nn.MultiLabelSoftMarginLoss()
train(XORNet, 1000, X_XOR, Y_XOR,criterio, optimizer)

In [ ]:
predicciones= XORNet(X_XOR)
print(predicciones)

### Ejercicio 2:

Encuentra un modelo para clasificar los datos contenidos en Xs y Ys, para lo cual debes encontrar el numero de capas y neuronas adecuado, así como una taza de aprendizaje.

HINT: Puedes utilizar [Playground.tensorflow](https://playground.tensorflow.org) para buscar gráficamente los modelos y al final implementarlos.

In [ ]:
def to_learn1(x,y):
    return 1 if x * y >= 0 else -1;

def asignar_color(clase):
    if(clase==1):
        return 'r'
    elif(clase==-1):
        return 'b'

# Observaciones
En este ejercicio se modificaron los conjuntos de datos por conveniencia, añadiendoles más elementos. Es posible que en el primer entrenamiento la exactitud de las predicciones disminuya hasta a un 80%. Por tanto, en caso de resultar así, en el primer entrenamiento, se recomienda volver a generar el conjunto de datos, el modelo y volverlo a entrenar. La tasa máxima de exactitud fue del 100%.

Por otro lado, dado el error **RuntimeError: expected scalar type Long but found float**, no se pudo utilizar la función de perdida **nn.BCELoss()**, se trato de resolver el problema con investigación y consultando a los ayudantes y profesora, pero no fue posible corregirlo. Esto mismo aplica para el ejercicio tres. De ahí la utilización de las funciones: **nn.MultiLabelSoftMarginLoss()** y **nn.SoftMarginLoss()**. Se considera que esta es la razón de la falta de ajuste en el entrenamiento.

In [ ]:
# Características de entrada y salida
Xs = [random.uniform(-5,5) for _ in range(0,200)] # Puede ajustar para más elementos, no menos
Ys = [random.uniform(-5,5) for _ in range(0,200)]
Zs = [to_learn1(punto[0],punto[1]) for punto in zip(Xs,Ys)]
Colores = [asignar_color(p) for p in Zs]
XYs = [[a, b] for a, b in zip(Xs, Ys)]
X_features1 = torch.Tensor(XYs)
Y_features1 = torch.tensor(Zs)
plt.scatter(Xs,Ys,c=Colores)
plt.plot()

In [ ]:
class Exercise2(nn.Module):
  def __init__(self):
    super(Exercise2,self).__init__()
    self.hidden_layer1 = nn.Linear(2, 8)
    self.hidden_layer2 = nn.Linear(8, 4)
    self.output_layer = nn.Linear(4, 1)
    self.act_func = nn.Tanh()
  
  def forward(self, x):
     x = self.hidden_layer1(x)
     x = self.act_func(x)
     x = self.hidden_layer2(x)
     x = self.act_func(x)
     x = self.output_layer(x)
     x = self.act_func(x)
     return x

In [ ]:
# Función que calcula de exactitud de un modelo
def accuracy_fn(y_true, y_pred):
  counter = 0
  for yt,yp in zip(y_true, y_pred):
    if yt==yp[0]:
      counter = counter+1
  return (counter/len(y_true))*100

In [ ]:
# Creamos al modelo, definimos la función de perdida y el optimizador, así como la tasa de aprendizaje
model1 = Exercise2()
criterio1 = nn.SoftMarginLoss()
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.01)
plot_decision_boundary(model1,X_features1, Y_features1)

In [ ]:
# Entrenamos al modelo
train(model1, 1000, X_features1, Y_features1, criterio1, optimizer1)

In [ ]:
# Probamos el modelo
with torch.inference_mode():
  predicciones1= model1(X_features1)
plot_decision_boundary(model1, X_features1, Y_features1)
print("Exactitud: ", accuracy_fn(Y_features1, predicciones1))

### Ejercicio 3:

Encuentra un modelo para clasificar los datos contenidos en X2s y Y2s, para lo cual debes encontrar el numero de capas y neuronas adecuado, así como una taza de aprendizaje.

# Observaciones
Este modelo pretende ajustarse al modelo que clasifica el conjunto de datos del playground. Como se menciono antes, es probable que el uso de las funciones de perdida antes mencionadas, en lugar de alguna otras más eficientes, para este tipo de conjunto de datos pueda indicar la disminución del 'aprendizaje'.

La tasa de exactitud máxima obtenida en las pruebas fue del 64%, y la mínima del 50% , lo cual se le atribuye a lo antes mencionado. En caso de requerir más información, puede consultar el archivo **readme.md**

In [ ]:
def to_learn2(deltaT,label):
    n=100
    Xs = []
    Ys = []
    Cs = [label]*n
    for i in range(0,n):
        r = i / n * 20;
        t = 1.3 * i / n * 2 * math.pi + deltaT;
        x = r * math.sin(t) + random.uniform(-1, 1);
        y = r * math.cos(t) + random.uniform(-1, 1);
        Xs.append(x)
        Ys.append(y)
    return Xs,Ys,Cs

def create_data():
    x1,y1,cs1 = to_learn2(0,-1)
    x2,y2,cs2 = to_learn2(math.pi,1)
    x1.extend(x2)
    y1.extend(y2)
    cs1.extend(cs2)
    colores=[asignar_color(p) for p in cs1]
    return x1,y1,cs1,colores

X1, X2,clases,colores = create_data()

In [ ]:
# Generamos los tensores para las características de entrada y salida
Y_features2 = torch.Tensor(clases)
X_features2 = torch.Tensor([[a, b] for a, b in zip(X1, X2)])

In [ ]:
class Exercise3(nn.Module):
  def __init__(self):
    super(Exercise3,self).__init__()
    self.hidden_layer1 = nn.Linear(7, 16)
    self.hidden_layer2 = nn.Linear(16, 8)
    self.output_layer = nn.Linear(8, 1)
    self.act_func = nn.Tanh()
  
  def forward(self, x):
    #print(x)
    if isMatrix(x.tolist()) :
     x = addFeaturesList(x)
    else:
     x = addFeaturesTuple(x)
    x = self.hidden_layer1(x)
    x = self.act_func(x)
    x = self.hidden_layer2(x)
    x = self.act_func(x)
    x = self.output_layer(x)
    x = self.act_func(x)
    return x

In [ ]:
# Creamos al modelo, definimos la función de perdida y el optimizador, así como la tasa de aprendizaje
model2 = Exercise3()
criterio2 = nn.MultiLabelSoftMarginLoss()
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.01)
#predicciones2 = model2(X_features2)
#print("exactitud: ", accuracy_fn(Y_features2, predicciones2))
plot_decision_boundary(model2,X_features2, Y_features2)

In [ ]:
# Entrenamos el modelo
train(model2, 2000, X_features2, Y_features2, criterio2, optimizer2)

In [ ]:
# Make predictions on the test data
with torch.inference_mode():
  predicciones2= model2(X_features2)
#print(predicciones2)
#print(Y_features2)
plot_decision_boundary(model1, X_features2, Y_features2)
print("exactitud: ", accuracy_fn(Y_features2, predicciones2))

In [ ]:
# Funcion que calcula las caracteríticas para una lista de listas de dos elementos
# X son las caracteristicas de entrada
def addFeaturesList(X):
  X_list = X.tolist()
  #print("Caract: ",X_list)
  X1 = [x[0] for x in X_list]
  X2 = [x[1] for x in X_list]
  X1_2 = list(map(lambda x: x ** 2, X1))
  X2_2 = list(map(lambda x: x ** 2, X2))
  X1X2 = list(map(lambda x, y: x * y, X1, X2))
  X1_sen = list(map(lambda x: math.sin(x), X1))
  X2_sen = list(map(lambda x: math.sin(x), X2))
  return torch.Tensor([[a, b, c, d, f, g, h] for a, b, c, d, f, g, h in zip(X1, X2, X1_2, X2_2, X1X2, X1_sen, X2_sen)])

# Funcion que calcula las caracteríticas para una lista de dos elementos
def addFeaturesTuple(X):
  X_list = X.tolist()
  X1 = X_list[0]
  X2 = X_list[1]
  X1_2 = X1 ** 2
  X2_2 = X2 ** 2
  X1X2 = X1*X2
  X1_sen = math.sin(X1)
  X2_sen = math.sin(X2)
  return torch.Tensor([X1, X2, X1_2, X2_2, X1X2, X1_sen, X2_sen])

In [20]:
# Función que determina si una entrada es una matriz
def isMatrix(X):
  if type(X) == list and all(isinstance(i, list) for i in X):
    return True
  else:
    return False